# Mecabを用いた形態素解析によるクリーニング

In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [ ]:
# データのパス
data_path = 'data/processed/android_cleaned.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

In [ ]:
# データのpositive_review, negative_reviewが「特になし」または「特になし。」を含む場合は，その欄を「-」に変換

df['positive_review'] = df['positive_review'].fillna('').apply(lambda x: '-' if '特になし' in x else x)
df['negative_review'] = df['negative_review'].fillna('').apply(lambda x: '-' if '特になし' in x else x)

# データのpositive_review, negative_reviewが欠損している or 空白の場合は，その欄を「-」に変換

df['positive_review'] = df['positive_review'].replace('', '-')
df['negative_review'] = df['negative_review'].replace('', '-')

df.head()

In [ ]:
!pip show mecab-python3

In [ ]:
import MeCab
print(MeCab.VERSION)


In [ ]:
import MeCab

# mecabrcのパスを明示的に指定
mecab = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")

# テキスト解析
print(mecab.parse("形態素解析を試してみます。"))


In [ ]:
review_document = 'このサイズ感が素晴らしいです。'
ochasen_tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")
node = ochasen_tagger.parse(review_document)

print(node)

In [ ]:
def lemmatize_entities(entity_list, ochasen_tagger):
    for i, entity in enumerate(entity_list):
        node = ochasen_tagger.parse(entity)
        lines = node.split("\n")
        for line in lines[:-2]:  # 最後のEOSや空行を無視
            fields = line.split("\t")
            print(fields)  # デバッグ用
            
            if len(fields) >= 2:
                # fields[1]をカンマで分割
                features = fields[1].split(",")
                # features[6]が原形（基本形）
                if len(features) > 6:
                    entity_list[i] = features[6]
    return entity_list

# 入力リスト
entity_list = ['強い', '強く', '強']

# 関数の実行
lemmatized_list = lemmatize_entities(entity_list, ochasen_tagger)
print("結果:", lemmatized_list)


In [ ]:
def extract_keywords(review_document, ochasen_tagger):
    review_keywords_list = []
    node = ochasen_tagger.parse(review_document)
    lines = node.split("\n")

    for line in lines:
        if line == "EOS" or line.strip() == "":
            continue
        fields = line.split("\t")
        if len(fields) < 2:
            continue

        pos_info = fields[1].split(",")
        pos_major = pos_info[0]  # 名詞,動詞,形容詞など
        pos_sub1 = pos_info[1]   # 固有名詞,一般 などの細分類

        if pos_major in ["名詞", "形容詞", "動詞"]:
            review_keywords_list.append(fields[0])

    return review_keywords_list

# Example usage
review_document = 'インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常に邪魔。'
keywords = extract_keywords(review_document, ochasen_tagger)
print(keywords)


In [ ]:
# データのpositive_review, negative_reviewに対して形態素解析を実行，結果を新たなカラムとして追加

df['positive_review_keywords'] = df['positive_review'].apply(lambda x: extract_keywords(x, ochasen_tagger))
df['negative_review_keywords'] = df['negative_review'].apply(lambda x: extract_keywords(x, ochasen_tagger))

df.head()

In [11]:
# データを保存

df.to_csv('data/processed/android_cleaned_mecab.csv', index=False)